# PROJET 10 DATA ANALYST

# OBJECTIF DE CE NOTEBOOK

Pour l'Organisation Nationale de lutte Contre le Faux-Monnayage (ONCFM), nous devons produire :

- Une analyse descriptive des données, notamment la répartition des dimensions des billets, le nombre de vrais / faux billets, etc.
- Une détection automatisée des faux billets à partir des dimensions de ces derniers. Les méthodes à utiliser sont la régression logistique et k-means avec une matrice de confusion pour évaluer les performances des modèles. Une fois la phase d'entrainement et de test achevée, l'algorithme devra être capable de prédire si un billet est vrai ou faux.

Glossaire :
- diagonal : la diagonale du billet (en mm)
- height_left : la hauteur du billet (mesurée sur le côté gauche, en mm)
- height_right : la hauteur du billet (mesurée sur le côté droit, en mm)
- length : la longueur du billet (en mm)
- margin_low : la marge entre le bord inférieur du billet et l'image de celui-ci (en mm)
- margin_up : la marge entre le bord supérieur du billet et l'image de celui-ci (en mm)

## Etape 1 - Importation des librairies et chargement des fichiers

## 1.1 - Importation des librairies

In [ ]:
#Importation des librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import NearestCentroid
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import scipy.stats as ss
from scipy.cluster.hierarchy import dendrogram, linkage

import statsmodels.api as sm

In [ ]:
#Chargement de la librairie graphique
sns.set()

## 1.2 - Chargement du fichier et séparation en 2 DataFrames

In [ ]:
#Importation du fichier population.csv en mettant l'index sur 'Zone'
billet = pd.read_csv('./Data_source/billets.csv', sep=';')

In [ ]:
#Affichage des dimensions et de leurs types
display(billet.info())

Nous pouvons voir que sur les 1500 lignes, la variable 'Height_left' contient des valeurs manquantes.

In [ ]:
#Affichage d'un échantillon
display(billet.sample(5))

In [ ]:
#Séparation en 2 DataFrames
billet_vrai = billet.loc[billet['is_genuine'] == True, :].copy()
billet_vrai.drop('is_genuine', axis=1, inplace=True)

billet_faux = billet.loc[billet['is_genuine'] == False, :].copy()
billet_faux.drop('is_genuine', axis=1, inplace=True)

## Etape 2 - Analyse exploratoire des données

## 2.2 - Statistiques descriptives

In [ ]:
#Affichage des statistiques descriptives
stats_descr = billet.describe().round(2)
display(stats_descr)

Nous pouvons remarquer 37 valeurs manquantes dans la variable 'margin_low'.

## 2.3 - Analyse univariée

In [ ]:
#Affichage des histogrammes avec la densité de probabilité
for col in stats_descr.columns:
    mu = stats_descr.loc['mean', col]
    sigma = stats_descr.loc['std', col]
    #Règle de Sturges pour déterminer approximativement le nombre optimal de classes
    num_bins = int(np.ceil(np.log2(stats_descr.loc['count', col])) + 1)
    #num_bins = 15
    print(num_bins)

    fig, ax = plt.subplots(figsize=(8, 5))

    #Affichage de l'histogramme
    n, bins, patches = ax.hist(billet[col], num_bins, density=True)

    #Affichage de la densité de probabilité
    y = ((1 / (np.sqrt(2 * np.pi) * sigma)) *
         np.exp(-0.5 * (1 / sigma * (bins - mu))**2))
    ax.plot(bins, y, '--')
    ax.set_xlabel('Valeurs')
    ax.set_ylabel('Densité de probabilité')
    ax.set_title(f"Distribution de '{col}' et densité de probabilité : "
                 fr'$\mu={mu:.2f}$, $\sigma={sigma:.2f}$')

    fig.tight_layout()
    plt.show()

    #Test de Kolmogorov-Smirnov
    print('Si p-value est inférieure à 0.05 alors on rejette H0 = normalité: {}\n\n'.format(ss.kstest(billet_vrai[col], 'norm')))

Les densités de probabilité des variables 'margin_low' et 'length' sont éloignées d'une loi normale principalement à cause de la présence des faux billets.

In [ ]:
#Affichage des boxplots
for col in stats_descr.columns:
    sns.boxplot(data=billet[col], orient='h')
    
    plt.title(f"Boxplot de '{col}'")
    plt.show()

Nous ne remarquons pas de valeurs aberrantes dans les variables.

## 2.4 - Analyse bivariée

**Matrice de corrélation entre les variables**

In [ ]:
# TODO : voir pour faire l'analyse bivariée sans les lignes avec des valeurs manquantes
#Matrice des corrélations utilisant le coefficient de corrélation de Pearson
corr_matrix = billet.iloc[:, 1:].corr(method='pearson', min_periods=20)

#Masque pour la partie triangulaire supérieure de la matrice
mask = np.triu(corr_matrix)

In [ ]:
#Heatmap représentant la matrice des corrélations
plt.figure(figsize=(15,6))
plt.title("Heatmap des coefficients de corrélation de Pearson entre les variables", fontsize=14)

sns.heatmap(corr_matrix, annot=True, vmin=-1, vmax=1, cmap='coolwarm', mask=mask, fmt='.2f')
plt.show()

Nous pouvons voir que les variables 'length' et 'is_genuine' sont très fortement corrélées (0.85). Ce qui serait une piste pour la détection des faux billets à l'aide de la regression logistique...

In [ ]:
#Pairplots pour visualiser les potentielles corrélations
sns.pairplot(billet, hue='is_genuine', corner=True)
plt.show()

## 2.5 - Imputation des valeurs manquantes

## 2.5.1 - Choix des variables explicatives pour la regression linéaire

In [ ]:
# TODO : voir si encore pertinent
#Conversion de 'is_genuine' en 0 et 1
billet['is_genuine'].replace([True, False], [1,0], inplace=True)

In [ ]:
#Séparation en deux DataFrames avec et sans NA
billet_isna = billet.loc[billet['margin_low'].isna(), :].copy()
billet_dropna = billet.dropna().copy()

In [ ]:
#Fonction permettant de sélectionner les variables explicatives les plus pertinentes à l'aide de la méthode backward regression
def backward_regression(X, y,
                        threshold_out = 0.05,
                        verbose=True):
    included=list(X.columns)
    while True:
        changed=False
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        pvalues = model.pvalues.iloc[1:] #On ne prend pas en compte la constante
        worst_pval = pvalues.max() # null si la p-value n'existe pas
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print("Drop '{}' car p-value {} > 0.05".format(worst_feature, round(worst_pval, 4)))
        if not changed:
            break
    print(model.summary())
    return included

In [ ]:
#Sélection des variables explicatives et de la variable à expliquer
y = billet_dropna['margin_low']
x = billet_dropna.drop(['is_genuine', 'margin_low'], axis=1)

In [ ]:
var_keep_list = backward_regression(x, y)
print('Liste des variables explicatives sélectionnées : {}'.format(var_keep_list))

## 2.5.2 - Sépaparation des données en train et test pour évaluation de la performance de la régression linéaire

In [ ]:
#Sélection des variables explicatives après backward regression et ajout de la constante
x = billet_dropna[var_keep_list].copy()
#x = sm.add_constant(x)
display(x.head(5))

#split des données en train et test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [ ]:
lm = LinearRegression()

scores = cross_val_score(lm, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

# RSME
np.sqrt(np.mean(np.absolute(scores)))

In [ ]:
# step-1: create a cross-validation scheme
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)

# step-2: specify range of hyperparameters to tune
hyper_params = [{'n_features_to_select': list(range(1, 6))}]
print(hyper_params)

# step-3: perform grid search
# 3.1 Recursive Feature Elimination (RFE) with linear regression model
lm = LinearRegression()
lm.fit(X_train, y_train)
rfe = RFE(lm)

# 3.2 call GridSearchCV()
model_cv = GridSearchCV(estimator = rfe,
                        param_grid = hyper_params,
                        scoring= 'r2',
                        cv = folds,
                        verbose = 1,
                        return_train_score=True)

# fit the model
model_cv.fit(X_train, y_train)

In [ ]:
# cv results
cv_results = pd.DataFrame(model_cv.cv_results_)
display(cv_results)

In [ ]:
# plotting cv results
plt.figure(figsize=(16,6))

plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_test_score"])
plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_train_score"])
plt.xlabel('number of features')
plt.ylabel('r-squared')
plt.title("Optimal Number of Features")
plt.legend(['test score', 'train score'], loc='upper left')

In [ ]:
# final model
n_features_optimal = 2

lm = LinearRegression()
lm.fit(X_train, y_train)

rfe = RFE(lm, n_features_to_select=n_features_optimal)
rfe = rfe.fit(X_train, y_train)

In [ ]:
# tuples of (feature name, whether selected, ranking)
# note that the 'rank' is > 1 for non-selected features
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

In [ ]:
# TODO : mettre une cross validation
#Instanciation et entrainment du modèle
model = sm.OLS(y_train, X_train).fit()

#Prédiction à partir des données de test
y_pred = model.predict(X_test)

In [ ]:
#Evaluation des performances du modèle avec RSME
print('RMSE : {}'.format(round(mean_squared_error(y_test, y_pred, squared=True), 2)))

Ça représente environ 10% de la moyenne de la variable 'margin_low'. Ce qui est acceptable.

## 2.5.3 - test des conditions de validité de la régression linéaire

In [ ]:
# TODO : faire les tests de validité de la régression linéaire avec graphique "Q-Q plot" et “standardized residuals vs fitted plot”

## 2.5.4 - Prediction des valeurs manquantes

In [ ]:
#Instanciation et entrainment du modèle
model = sm.OLS(y, x).fit()

#Prédiction des valeurs manquantes
billet_isna['margin_low'] = model.predict(sm.add_constant(billet_isna[var_keep_list])).round(2)

display(billet_isna.head(10))

## Etape 3 - Split des données et Centrage, Reduction (Scaling)

**- Split des données**

In [ ]:
#Conversion de dispo_alim en numpy array et affichage
x = billet.values

#Insertion de 'Zone' (pays) dans 'names'
names = billet.index

#Insertion des noms de colonne dans 'features'
features = billet.columns

**- Centrage et Réduction**

In [ ]:
#Instanciation du Scaler
std_scale = preprocessing.StandardScaler()

#Entrainement
std_scale.fit(x)

In [ ]:
#Transformation
x_scaled = std_scale.transform(x)

In [ ]:
#Pour vérifier que le centrage/réduction s'est bien passé
pd.DataFrame(x_scaled).describe().round(2).iloc[1:3:, : ]